## 텐서플로를 사용한 신경망 훈련

### 텐서플로 설치

In [1]:
import tensorflow as tf
import numpy as np
print('텐서플로 버전:', tf.__version__)

2025-06-04 04:44:58.809009: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 04:44:59.004677: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 04:44:59.111814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749012299.250524   16840 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749012299.290743   16840 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749012299.592405   16840 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

텐서플로 버전: 2.19.0


In [2]:
print(tf.keras.Sequential)

<class 'keras.src.models.sequential.Sequential'>


### 파이썬 버전 충돌할 때 가상환경설치
- 파이썬 버전 3.12 다운로드
- 터미널에서 py --list
- py -3.12 -m venv (가상환경이름)
- Scripts 폴더에서 activate
- pip install 명령은 Scripts폴더에서

### 텐서플로에서 텐서 만들기

In [3]:
a = np.array([1, 2, 3], dtype=np.int32)
b = [4, 5, 6]

t_a = tf.convert_to_tensor(a)
t_b = tf.convert_to_tensor(b)

print(t_a)
print(t_b)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor([4 5 6], shape=(3,), dtype=int32)


2025-05-28 05:00:12.220182: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
tf.is_tensor(a), tf.is_tensor(t_a)

(False, True)

In [ ]:
t_ones = tf.ones((2, 3))

t_ones.shape

In [ ]:
t_ones.numpy()


In [ ]:
const_tensor = tf.constant([1.2, 5, np.pi], dtype=tf.float32)

print(const_tensor)

In [ ]:
tf.fill((2, 3), 1)


In [ ]:
tf.one_hot([0, 1, 2], 4)


### 텐서의 데이터 타입과 크기 조작하기

In [ ]:
t_a_new = tf.cast(t_a, tf.int64)

print(t_a_new.dtype)

In [ ]:
t = tf.random.uniform(shape=(3, 5))

t_tr = tf.transpose(t)
print(t.shape, ' --> ', t_tr.shape)

In [ ]:
t = tf.zeros((30,))

t_reshape = tf.reshape(t, shape=(5, 6))

print(t_reshape.shape)

### 텐서에 수학 연산 적용하기

In [ ]:
tf.random.set_seed(1)

t1 = tf.random.uniform(shape=(5, 2), 
                       minval=-1.0,
                       maxval=1.0)

t2 = tf.random.normal(shape=(5, 2), 
                      mean=0.0,
                      stddev=1.0)

In [ ]:
t3 = tf.multiply(t1, t2).numpy()
print(t3)

In [ ]:
t4 = tf.math.reduce_mean(t1, axis=0)

print(t4)

In [ ]:
t5 = tf.linalg.matmul(t1, t2, transpose_b=True)

print(t5.numpy())

In [ ]:
t6 = tf.linalg.matmul(t1, t2, transpose_a=True)

print(t6.numpy())

## 텐서플로 데이터셋 API(tf.data)를 사용하여 입력 파이프라인 구축하기

### 텐서에서 텐서플로 데이터셋 만들기

In [ ]:
a = [1.2, 3.4, 7.5, 4.1, 5.0, 1.0]

ds = tf.data.Dataset.from_tensor_slices(a)

print(ds)

In [ ]:
for item in ds:
    print(item)

### 두 개의 텐서를 하나의 데이터셋으로 연결하기

In [ ]:
tf.random.set_seed(1)

t_x = tf.random.uniform([4, 3], dtype=tf.float32)
t_y = tf.range(4)

In [ ]:
ds_x = tf.data.Dataset.from_tensor_slices(t_x)
ds_y = tf.data.Dataset.from_tensor_slices(t_y)
    
ds_joint = tf.data.Dataset.zip((ds_x, ds_y))

for example in ds_joint:
    print('  x: ', example[0].numpy(), 
          '  y: ', example[1].numpy())

In [ ]:
## 방법 2:
ds_joint = tf.data.Dataset.from_tensor_slices((t_x, t_y))

for example in ds_joint:
    print('  x: ', example[0].numpy(), 
          '  y: ', example[1].numpy())

### shuffle(), batch(), repeat() 메서드

In [ ]:
tf.random.set_seed(1)

ds = ds_joint.shuffle(buffer_size=len(t_x))

print(t_x.numpy())

for example in ds:
    print('  x: ', example[0].numpy(), 
          '  y: ', example[1].numpy())

In [ ]:
ds = ds_joint.batch(batch_size=3)

batch_x, batch_y = next(iter(ds))

print('배치 x: \n', batch_x.numpy())

print('배치 y:   ', batch_y.numpy())

In [ ]:
ds = ds_joint.batch(3, drop_remainder=False).repeat(count=2)

for i,(batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

In [ ]:
tf.random.set_seed(1)

## 순서 1: shuffle -> batch -> repeat
ds = ds_joint.shuffle(4).batch(2).repeat(3)

for i,(batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

In [ ]:
tf.random.set_seed(1)

## 순서 1: shuffle -> batch -> repeat
ds = ds_joint.shuffle(4).batch(2).repeat(20)

for i,(batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

In [ ]:
tf.random.set_seed(1)

## 순서 2: batch -> shuffle -> repeat
ds = ds_joint.batch(2).shuffle(4).repeat(3)

for i,(batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

In [ ]:
tf.random.set_seed(1)

## 순서 2: batch -> shuffle -> repeat
ds = ds_joint.batch(2).shuffle(4).repeat(20)

for i,(batch_x, batch_y) in enumerate(ds):
    print(i, batch_x.shape, batch_y.numpy())

<br>
<br>
<br>

## 텐서플로로 신경망 모델 만들기 - 텐서플로 케라스 API (tf.keras)

## 붓꽃 데이터셋을 분류하는 다층 퍼셉트로 만들기

In [ ]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# 1. Iris 데이터 불러오기
iris = load_iris()
X = iris.data         # shape: (150, 4)
y = iris.target       # shape: (150,)

# 2. 섞어서 train/test로 분할 (80%/20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True
)

# 3. NumPy 배열을 Tensor로 변환
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.int32)

X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.int32)

# 4. 결과 확인
print("X_train_tf:", X_train_tf.shape, X_train_tf.dtype)
print("y_train_tf:", y_train_tf.shape, y_train_tf.dtype)


### 데이터셋 만들기

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train_tf, y_train_tf))
test_ds = tf.data.Dataset.from_tensor_slices((X_test_tf, y_test_tf))

# 5. 셔플 + 배치 + 프리페치 (훈련에 적합하게 구성)
train_ds = train_ds.shuffle(buffer_size=105).batch(16).repeat()
test_ds = test_ds.batch(16)

# 6. 결과 확인
for features, label in train_ds.take(1):
    print("입력 배치:", features.shape)
    print("레이블 배치:", label.shape)

In [ ]:
iris_model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='sigmoid', 
                          name='fc1', input_shape=(4,)),
    tf.keras.layers.Dense(3, name='fc2', activation='softmax')])

iris_model.summary()

In [ ]:
iris_model.compile(optimizer=tf.keras.optimizers.Adam(),
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

In [ ]:
import numpy as np

num_epochs = 100
training_size = 105
batch_size = 16
steps_per_epoch = int(np.ceil(training_size / batch_size))

history = iris_model.fit(train_ds, epochs=num_epochs, 
                        steps_per_epoch = steps_per_epoch, verbose=0)

In [ ]:
import matplotlib.pyplot as plt

hist = history.history

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(1, 2, 1)
ax.plot(hist['loss'], lw=3)
ax.set_title('Training loss', size=15)
ax.set_xlabel('Epoch', size=15)
ax.tick_params(axis='both', which='major', labelsize=15)

ax = fig.add_subplot(1, 2, 2)
ax.plot(hist['accuracy'], lw=3)
ax.set_title('Training accuracy', size=15)
ax.set_xlabel('Epoch', size=15)
ax.tick_params(axis='both', which='major', labelsize=15)
plt.tight_layout()
plt.show()

### 테스트 데이터셋에서 훈련된 모델 평가하기

In [ ]:
results = iris_model.evaluate(test_ds, verbose=0)
print('테스트 손실: {:.4f}   테스트 정확도: {:.4f}'.format(*results))

### 훈련된 모델 저장하고 복원하기

In [ ]:
iris_model.save('my_model.keras')  


In [ ]:
iris_model_new = tf.keras.models.load_model('my_model.keras')

iris_model_new.summary()

In [ ]:
results = iris_model_new.evaluate(test_ds, verbose=0)
print('테스트 손실: {:.4f}   테스트 정확도: {:.4f}'.format(*results))